# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
?prefilter_items

In [4]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Prefilter success: Decreased number of items from 83685 to 5001
Decreased # items from 83685 to 5001


In [5]:
recommender = MainRecommender(data_train_lvl_1)

In [6]:
recommender.get_als_recommendations(2375, N=5)

[899624, 1044078, 5569230, 871756, 844179]

In [7]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [8]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 1044078, 999270, 1115576, 15778319]

In [9]:
recommender.get_similar_users_recommendation(2375, N=5)

[9553048, 1101502, 1012801, 10457044, 974265]

In [10]:
user = 2375
pd.DataFrame({
    'get_als_recommendations': [recommender.get_als_recommendations(user, N=5)],
    'get_own_recommendations': [recommender.get_own_recommendations(user, N=5)],
    'get_similar_items_morecommendation': [recommender.get_similar_items_recommendation(user, N=5)],
    'get_similar_users_recommendation': [recommender.get_similar_users_recommendation(user, N=5)]
}).T

,0
get_als_recommendations,"[899624, 1044078, 5569230, 871756, 844179]"
get_own_recommendations,"[948640, 918046, 847962, 907099, 873980]"
get_similar_items_morecommendation,"[1046545, 1044078, 999270, 1115576, 15778319]"
get_similar_users_recommendation,"[9553048, 1101502, 1012801, 10457044, 974265]"


### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

In [11]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [12]:
result_lvl_1[result_lvl_1.user_id == 2375].T

,2044
user_id,2375
actual,"[835300, 848029, 862714, 896613, 923746, 93246..."


In [13]:
%%time
result_lvl_1['als_rec-s'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(user=x, N=50))

IndexError: index 2496 is out of bounds for axis 0 with size 2496

In [14]:
%%time
result_lvl_1['own_rec-s'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(user=x, N=50))

ValueError: userid is out of bounds of the user_items matrix

In [15]:
%%time
result_lvl_1['similar_items_rec-s'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(user=x, N=50))

Wall time: 20.7 s


In [16]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""

    popular = data.groupby(by='item_id')['sales_value'].sum().reset_index()
    popular.sort_values(by='sales_value', ascending=False, inplace=True)

    recs = popular.head(n).item_id

    return recs.tolist()

In [17]:
%%time
result_lvl_1['top-popular'] = result_lvl_1['user_id'].apply(lambda x: popularity_recommendation(data_train_lvl_1, n=50))

Wall time: 1min 43s


In [18]:
result_lvl_1['own+top'] = result_lvl_1['own_rec-s'] + result_lvl_1['top-popular']

KeyError: 'own_rec-s'

In [ ]:
result_lvl_1[result_lvl_1.user_id == 2375].T

In [ ]:
def score_recomends(result_df):
    import pandas as pd
    from src.metrics import precision_at_k, recall_at_k
    df = pd.DataFrame(
        [
            (model_,
             round(result_df.apply(lambda row: precision_at_k(recommended_list=row[str(model_)], bought_list=row['actual']), axis=1).mean(), 5),
             round(result_df.apply(lambda row: recall_at_k(recommended_list=row[str(model_)], bought_list=row['actual']), axis=1).mean(), 5),
             ) for model_ in list(result_df)[2:]
        ], columns=['model', 'precision_at_k', 'recall_at_k']
    ).sort_values(by='precision_at_k', ascending=False)

    return df

In [ ]:
score_recomends(result_df=result_lvl_1)

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  

In [ ]:
result_lvl_1_var_2 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index().rename(columns={'item_id': 'actual'})

In [ ]:
result_lvl_1_var_2.head(2).T

In [ ]:
%%time
for k in [20, 50, 100, 200, 500]:
    print(f'k={k}')
    result_lvl_1_var_2[f'similar_items_k={k}'] = \
        result_lvl_1_var_2['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(user=x, N=k))
print('The End')

In [ ]:
result_lvl_1_var_2.head(1).T

In [ ]:
k=500
result_lvl_1_var_2.apply(
    lambda row: recall_at_k(recommended_list=row[f'similar_items_k={k}'], bought_list=row['actual'], k=k), axis=1
).mean()

In [ ]:
recals_at_list_k = pd.DataFrame(
    [
        (
            k, 
            result_lvl_1_var_2.apply(
                lambda row: recall_at_k(
                    recommended_list=row[f'similar_items_k={k}'], bought_list=row['actual'], k=k), axis=1).mean()
        ) for k in [20, 50, 100, 200, 500]
        
    ], columns={'k', 'value'}).sort_values(by='value', ascending=False)

recals_at_list_k

In [ ]:
plt.plot(recals_at_list_k['k'],
         recals_at_list_k['value'])
plt.grid();

In [ ]:
recals_at_list_k.head(1)

C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?

recall при k=500 показал наибольшее значениe

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [ ]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

#создание списков кандидатов на певом уровне
users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=500))

#разворачивание списков в столбцы
df = pd.DataFrame({'user_id':users_lvl_2.user_id.values.repeat(len(users_lvl_2.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2.candidates.values).astype('int64')})

In [ ]:
#создание таргета на основе полученных на первом уровне кандидатов и совпадения их с валидационным датасетом для первого уровня
targets_lvl_2 = data_train_lvl_2.drop_duplicates(['user_id', 'item_id'])[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 
targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2['target'].mean()

In [ ]:
# заменим бренд СТМ на 1, остальное на 0
item_features['brand'] = item_features['brand'].map({'National': 0, 'Private': 1})

In [ ]:
# Заменимм возраст по нижней границе
user_features['age_desc'] = user_features['age_desc'].map({'45-54': 45, '35-44': 35, '55-64': 55, '25-34': 25, '65+': 65, '19-24': 19})

# Заменимм доход по нижней границе
user_features['income_desc'] = user_features['income_desc'].map({
    '50-74K': 50000, '35-49K': 35000, '75-99K': 75000, '25-34K': 25000, '15-24K': 15000, 'Under 15K': 0, 
    '125-149K': 125000, '100-124K': 100000, '150-174K': 150000, '250K+': 250000, '175-199K': 175, 
    '200-249K': 200000})
# Заменим в кол-ве детей 3+ на 3
user_features['kid_category_desc'] = user_features['kid_category_desc'].map({'None/Unknown': 0, '3+': 3})

# Заменим собственость на "вероятность владения домом"
user_features['homeowner_desc'] = user_features['homeowner_desc'].map({'Homeowner': 1, 'Unknown': 0, 'Renter': 0, 
                                    'Probable Owner': 0.75, 'Probable Renter': 0.25})
#  Заменим в размере дома 5+ на 5
user_features['household_size_desc'] = user_features['household_size_desc'].map({'5+': 5})

In [ ]:
# Добавим средний чек
user_features = user_features.merge(right = user_features.merge(right=data, on='user_id', how='left') \
                                    .groupby(by='user_id')['sales_value'].mean() \
                                    .reset_index().rename(columns={'sales_value': 'average_purchase'}),
                                    on='user_id', how='left')

In [ ]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

In [ ]:
X_train = targets_lvl_2.drop(['user_id', 'item_id', 'target'], axis=1)
y_train = targets_lvl_2['target']

In [ ]:
X_train

In [ ]:
# cat_feats = X_train.columns[1:].tolist()
cat_feats = ['department', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product', 'marital_status_code', 'hh_comp_desc']
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [ ]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

In [ ]:
train_preds = lgb.predict_proba(X_train)[:, 1]

In [ ]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique() \
            .reset_index().rename(columns={'item_id': 'actual'})

In [ ]:
result_lvl_2.head(2)

In [ ]:
temp_result = targets_lvl_2.loc[:, ['user_id', 'item_id']]

In [ ]:
#добавление неотсортированных кандидатов
candidats = temp_result.groupby('user_id')['item_id'].apply(list)
result_lvl_2 = result_lvl_2.merge(candidats, on=['user_id'], how='left').rename(columns={'item_id': 'own_recs'})

In [ ]:
#добавление отсортированных по LGBMClassifier кандидатов
temp_result['train_preds'] = train_preds
temp_result.sort_values('train_preds', ascending=False, inplace=True)

top_candidats = temp_result.groupby(by='user_id')['item_id'].apply(list)
result_lvl_2 = result_lvl_2.merge(right=top_candidats, on=['user_id'], how='left')
result_lvl_2 = result_lvl_2.rename(columns={'item_id': 'lgbm_recs'})

In [ ]:
result_lvl_2.head(3)

In [ ]:
result_lvl_2.loc[result_lvl_2['lgbm_recs'].isnull()]

In [ ]:
result_lvl_2 = result_lvl_2.drop(np.where(result_lvl_2['lgbm_recs'].isnull())[0], axis=0)

In [ ]:
result_lvl_2

In [ ]:
result_lvl_2.apply(lambda row: precision_at_k(row['lgbm_recs'], row['actual']), axis=1).mean()

In [ ]:
result_lvl_2.apply(lambda row: precision_at_k(row['own_recs'], row['own_recs']), axis=1).mean()

### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 0.27%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 